In [1]:
from bs4 import BeautifulSoup
import urllib 
import pandas as pd
import requests
import re
import urllib.request
import webbrowser
from selenium import webdriver
import time
import string
import numpy as np
from bs4.element import Tag
from bs4.dammit import EncodingDetector

internal_only = True #parameter which determines whether we will restrict software scraping to internal URLs, ie they have the parent domain
scrape_login_links = True #set to true to scrape URLs found on login pages also

In [2]:
import io
import shutil
'''
with io.open('cupola urls.csv', encoding='utf-8', errors='ignore') as source:
    with io.open('cupola urls_utf.csv', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

df = pd.read_csv('cupola urls_utf.csv')
'''

with io.open('Tech survey test4.csv', encoding='utf-8', errors='ignore') as source:
    with io.open('Tech survey test4_utf.csv', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

df = pd.read_csv('Tech survey test4_utf.csv')

In [3]:
df

,Unnamed: 0,organization_id,name,url,batch
0,0,1,ATSSA,https://www.atssa.com/,1
1,1,8,The national Council,https://www.thenationalcouncil.org/,2


In [4]:
#dictionaries and suppression lists
exclude_keywords= [
    'twitter',
    'facebook',
    'youtube',
    'youtu.be',
    'pinterest',
    'instagram',
    'linkedin',
    'paypal',
    'venmo',
    'javascript',
    '@', 
    'google',
    'flickr'
]

#note exceptions to keyword rules here
exceptions_dict = {
    'iMIS':['optimist','maximise','optimism','optimised'],
    'Amilia':['familiar','unfamiliar','familial','familiarity'],
    'Tendenci':['tendencies'],
    'The Associate':['the associated'],
}

In [5]:
import io
import shutil

with io.open('software lkup v1.02.csv', encoding='utf-8', errors='ignore') as source:
    with io.open('software list_utf.csv', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

software = pd.read_csv('software list_utf.csv')

In [6]:
software

,software,software2,vendor_id,service_category_id,service_category
0,123 Signup,123Signup,235614.0,12.0,AMS
1,123Signup,123Signup,235614.0,12.0,AMS
2,ACGI PAGE,Association Anywhere,235619.0,12.0,AMS
3,AdvantageCS,AdvantageCS,235589.0,12.0,AMS
4,Alliance by Protech,Alliance,235590.0,12.0,AMS
...,...,...,...,...,...
109,WildApricot,Wild Apricot,235611.0,12.0,AMS
110,xCatalyst,xCatalyst,235674.0,12.0,AMS
111,Your People,Your People,235675.0,12.0,AMS
112,YourMembership,YourMembership,235612.0,12.0,AMS


In [7]:
d = pd.Series(software.software2.values,index=software.software).to_dict()
d

{'123 Signup': '123Signup',
 '123Signup': '123Signup',
 'ACGI PAGE': 'Association Anywhere',
 'AdvantageCS': 'AdvantageCS',
 'Alliance by Protech': 'Alliance',
 'Altai': 'Altai',
 'Amilia': 'Amilia',
 'Amilia.com': 'Amilia',
 'AMPAC': 'AMPAC',
 'Aptify': 'Aptify',
 'Association Online': 'Association Online ',
 'BenchPrep': 'BenchPrep',
 'Bitrix 24': 'Bitrix24',
 'Bitrix24': 'Bitrix24',
 'Blue Sky eLearn': 'Blue Sky eLearn',
 'BlueSkyeLearn': 'Blue Sky eLearn',
 'Brightspace': 'Brightspace',
 'Broad Point Engage': 'BroadPoint Engage',
 'BroadPoint Engage': 'BroadPoint Engage',
 'BroadPointEngage': 'BroadPoint Engage',
 'Cadmium': 'CadmiumCD',
 'CadmiumCD': 'CadmiumCD',
 'Classroom 24-7': 'Classroom 24/7',
 'Classroom24-7': 'Classroom 24/7',
 'ClearVantage': 'ClearVantage',
 'ClubExpress': 'ClubExpress',
 'cobalt.saas': 'Cobalt',
 'CourseStage': 'CourseStage',
 'Crowd Wisdom': 'Crowd Wisdom',
 'CrowdWisdom': 'Crowd Wisdom',
 'Cvent': 'Cvent',
 'Elevate LMS': 'Elevate LMS',
 'ElevateLMS':

In [8]:
batches = list(df.batch.drop_duplicates())

In [9]:
#Change this number to correspond with the number of batches you are running. 
list = range(0,301)

In [10]:
original_time = time.time()
batch_files = []

#column where we'll store the total links found per org
df['total_links'] = ''

for batch in batches:
    print('Starting batch ' + str(batch))
    
    start_time = time.time()
    step_time = time.time()
    filename = 'software_' + str(batch) + '.csv'
    batch_files.append(filename)

    batch_df = df[df.batch == batch]
    batch_df['url'] = batch_df['url'].str.lower()

    #clean urls to standard format
    clean_urls = []
    for url in batch_df['url']: 
        if 'http://www.' in url:
            clean_urls.append(url)
        elif 'https://www.' in url:
            clean_urls.append(url)
        elif ('www.' in url) & ('http://' not in url) & ('https://' not in url):
            clean_urls.append('http://' + url )
        else: 
            clean_urls.append('http://www.' + url)
    batch_df['clean_urls'] = clean_urls
    
    #derive domain root from clean url
    domain_find = []
    domain = []
    for clean_url in batch_df['clean_urls']:
        if 'https://' in url:
            domain_find.append(clean_url[12:])
        else:
            domain_find.append(clean_url[11:])

    for item in domain_find:
        root = item.find('.')
        r = item[:root]
        domain.append(r)

    batch_df['domain'] = domain
    
    domain_dict = pd.Series(batch_df.domain.values,index=batch_df.organization_id).to_dict()
    
    #this prevents files from downloading while the scraper visits sites, 
    #make sure chromedriver is compatible with your chrome version
    
    options = webdriver.ChromeOptions()
    
    prefs = {
    'download_restrictions': 3,
    }
    options.add_experimental_option(
    'prefs', prefs
    )
    
    
    #compile links to scrape
    driver = webdriver.Chrome('C:/Users/dbell/Data Science [Github]/chromedriver.exe', options = options)

    links = []
    o_id = []
    for index, row in batch_df.iterrows():
        try:
        
            driver.get(row['clean_urls'])
            o_id.append(row['organization_id'])
            links.append(row['clean_urls'])

            for a in driver.find_elements_by_xpath('.//a'):
                try:
                    link = a.get_attribute('href')
                    if link is not None:
                        #excluding bad rows
                        if (isinstance(link, float) is False) & (len(str(link))>5):
                            e=0
                            for w in exclude_keywords:
                                if w in link:
                                    e=e+1
                            if e==0: #there are no bad keywords present
                                links.append(link)
                                o_id.append(row['organization_id'])
                except:
                    a
        except:
            index

    driver.close()

    links_df = pd.DataFrame()
    links_df['organization_id'] = o_id
    links_df['link'] = links
    links_df.drop_duplicates(inplace=True)
    
    if scrape_login_links == False:
        all_links = links_df.copy()
    
    print("\tlinks scraped from main URLs --- %s seconds ---" % round(time.time() - step_time, 2))
    
    if scrape_login_links == True:
        step_time = time.time()

        #appending any links present on login pages, as these have a higher likelihood for having software tags

        driver = webdriver.Chrome('C:/Users/dbell/Data Science [Github]/chromedriver.exe', options = options)

        login_links = []
        login_o_id = []
        #iterate through all collected websites so far to see if they contain the word "password"; collect URLs if so
        for index, row in links_df.iterrows():
            if (internal_only == False) | ((internal_only == True) & (domain_dict[row['organization_id']] not in str(row['link']).lower())):
                try:
                    driver.get(row['link'])
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    string_soup = str(soup).lower()

                    if 'password' in string_soup:
                        for a in driver.find_elements_by_xpath('.//a'):
                            try:
                                link = a.get_attribute('href')
                                if link is not None:
                                    #excluding bad rows
                                    if (isinstance(link, float) is False) & (len(str(link))>5):
                                        e=0
                                        for w in exclude_keywords:
                                            if w in link:
                                                e=e+1
                                        if e==0: #there are no bad keywords present
                                            login_links.append(link)
                                            login_o_id.append(row['organization_id'])
                            except:
                                a
                except:
                    index

        driver.close()

        login_links_df = pd.DataFrame()
        login_links_df['organization_id'] = login_o_id
        login_links_df['link'] = login_links
        #merge the original set of links plus any login page URLs together
        all_links = pd.concat([links_df,login_links_df])
        all_links.drop_duplicates(inplace=True)

        print("\tlinks scraped from login URLs --- %s seconds ---" % round(time.time() - step_time, 2))
    
    step_time = time.time()
    
    #fill in the total_links column in df
    for index, row in df.iterrows():
        if row['organization_id'] in batch_df.organization_id.values:
            try: 
                df.at[index,'total_links'] = all_links[all_links.organization_id == row['organization_id']].shape[0]
            except:
                df.at[index,'total_links'] = 0
    
    #here we are identifying all org "internal" links collected
    internal_links = []
    for index, row in all_links.iterrows():
        if domain_dict[row['organization_id']] in str(row['link']).lower():
            internal_links.append(1)
        else:
            internal_links.append(0)

    all_links['internal_link'] = internal_links
    #all_links.to_csv('all links duncan test.csv')
    
    #iterate through each link to scrape software references
    org_ids = []
    ams_tags = []
    link = []
    internal = []

    #determine the scope based on internal_only parameter
    if internal_only:
        scrape_links = all_links[all_links.internal_link == 1]
    else:
        scrape_links = all_links.copy()
    
    driver = webdriver.Chrome('C:/Users/dbell/Data Science [Github]/chromedriver.exe', options = options)

    #we're now ready to search for software keywords in the HTML of our universe of URLs, these are stored in a dataframe
    for index, row in scrape_links.iterrows(): 
        try:
            driver.get(row['link'])
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            string_soup = str(soup).lower()
            for k in d:
                if str(k).lower() in string_soup:
                    if k in exceptions_dict:
                        include = True
                        #need to loop through the list of exception words here
                        for e in exceptions_dict[k]:
                            if e in string_soup:
                                include = False
                        if include:
                            org_ids.append(row['organization_id'])
                            link.append(row['link'])
                            ams_tags.append(d[k])
                            internal.append(row['internal_link'])
                    else:
                        org_ids.append(row['organization_id'])
                        link.append(row['link'])
                        ams_tags.append(d[k]) 
                        internal.append(row['internal_link'])

        except:
            index

    driver.close()
    
    print("\tsoftware scraped --- %s seconds ---" % round(time.time() - step_time, 2))

    software = pd.DataFrame()
    #software['batch'] = batch
    software['organization_id'] = org_ids
    software['source_link'] = link
    software['software'] = ams_tags
    software['internal_link'] = internal
    
    software.to_csv(filename, index=False)
    print("Batch time elapsed --- %s seconds ---" % (time.time() - start_time))
print("Overall process time elapsed --- %s seconds ---" % (time.time() - original_time))

Starting batch 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

	links scraped from main URLs --- 16.39 seconds ---
	links scraped from login URLs --- 12.03 seconds ---
	software scraped --- 318.75 seconds ---
Batch time elapsed --- 347.1958327293396 seconds ---
Starting batch 2
	links scraped from main URLs --- 31.02 seconds ---
	links scraped from login URLs --- 45.22 seconds ---
	software scraped --- 401.41 seconds ---


PermissionError: [Errno 13] Permission denied: 'software_2.csv'

In [ ]:
exceptions_dict['iMIS']

# Next Steps/ Development Ideas

Here I want to concat all of the software outputs and do the following:
    * dedup on org id and software
    * do some type of formula to predict the "confidence" of the software prediction by (idea based on spot check finds):
        * creating a point system for key type (keyword, code line)
        * count the amount of unique pages each software is found for an org id 
        * and maybe count how many times the string that key in contained in was found in ALL urls
        
    * Write code that has output in link_org_vendor form

In [ ]:
#This puts all output files together, needs to improve bcuz it pulls all column headers

all_software = pd.DataFrame(columns=['organization_id','source_link','software', 'internal_link'])
for file in batch_files:
    file_df = pd.read_csv(file)
    all_software = pd.concat([all_software,file_df])
    all_software.reset_index(inplace=True)
    try:
        all_software.drop(columns=['index'],inplace=True)
    except:
        pass
    try:
        all_software.drop(columns=['level_0'],inplace=True)
    except:
        pass
    

'''
#old way
#This is used so that range is customized based on dataframe
min = df.batch.min()
max = df.batch.max()

fout = open ('out.csv','a')
# first file:
for line in open('software_1.csv'):
    fout.write(line)
# now the rest:    
for num in range((min + 1), (max + 1)):
    f = open('software_'+str(num)+'.csv')
    next(f) # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

all_software = pd.read_csv('out.csv')
#all_software.drop_duplicates(inplace=True)

'''

In [ ]:
all_software.loc[39]['source_link']

In [ ]:
all_software

In [ ]:
# I will use this to see how many times a software is found related to each org
all_software_grouped = all_software.drop(columns=['internal_link'])
all_software_grouped= all_software_grouped.groupby(['organization_id', 'software']).count()
all_software_grouped

In [ ]:
all_software_grouped.reset_index(inplace=True)
merged = pd.merge(df,all_software_grouped,how='left',left_on='organization_id',right_on='organization_id')

#some code we can use to create and maintain a 'master' table of all software assigned by the scraper
#should add date software discovered
try:
    old_master = pd.read_csv('master.csv')
    master = old_master.concat([old_master,merged])
    master = master.drop_duplicates()
except:
    merged.to_csv('master.csv',index=False)

In [ ]:
merged